## Multiple linear regression
This code runs multiple linear regression within object-selective epoch defined based on ANOVA result. 

Response-selective neurons were filtered out by nested regression.

In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

from scipy import stats
from scipy.ndimage import gaussian_filter
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

import matplotlib as mpl
import matplotlib.pyplot as plt  

from datetime import date
import time

import random

from joblib import Parallel, delayed

import h5py

In [2]:
# no top and right spines in all plots
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

In [3]:
mother_path = Path('D:/Multi-modal project/')

### Parameter setting

In [4]:
num_iter = 1000

gauss_sigma = 2

# colors for multimodal, vis-only, aud-only conditions
color = ['mediumorchid','cornflowerblue','lightcoral','gray']
color2 = ['cyan','magenta','brown']
linestyle = ['-',':']

today = str(date.today())

### Data preparation

In [5]:
save_path = mother_path /'analysis'/'result'/'3.1 Multiple linear regression'/today
cell_path = mother_path/'analysis'/'result'/'zFR export'/'13-Apr-2022 (5 trials)'
data_path = mother_path /'analysis'/'result'/'3. ANOVA'

cell_list = os.listdir(cell_path)

# load hdf5 files containing shuffled results
f = h5py.File(data_path/'2023-05-30'/'2023-05-30_ANOVA_result.hdf5','r')

# make hdf5 save file
os.makedirs(save_path,exist_ok=True)
s = h5py.File(save_path/f'{today}_multiple_regression_result.hdf5','w')

# Data analysis

In [6]:
def MLR(fr,x1,x2,shuffle):
    """
    This function performs multiple linear regression to estimate firing rates within
    object-selective epoch
    """
    # trial condition shuffling
    if shuffle:
        state = np.random.get_state()
        x1 = np.random.permutation(x1)
        np.random.set_state(state)
        x2 = np.random.permutation(x2)
    
    lr1 = OLS(fr,add_constant(x1)).fit()
    lr2 = OLS(fr,add_constant(x2)).fit()
    
    beta_coef, beta_coef_choice = lr1.params[1:], lr2.params[1:]

    AIC, AIC_choice = lr1.aic, lr2.aic
    BIC, BIC_choice = lr1.bic, lr2.bic
    
    if shuffle==0:
        pval, pval_choice = lr1.pvalues[1:], lr2.pvalues[1:]    
    else:
        pval, pval_choice = 0, 0
        
    return beta_coef, beta_coef_choice, pval, pval_choice, AIC, AIC_choice, BIC, BIC_choice

In [7]:
def plot_SDF_beta(df,linewidth,smooth,save,save_format):
    """
    This function plots mean firing rate patterns of each stimulus condition
    and beta coefficients for visual and auditory terms in multiple linear regression.
    """
    cond = [(df.Type=='Multimodal')&(df.RWD_Loc==boy_goal),
            (df.Type=='Multimodal')&(df.RWD_Loc==egg_goal),
            (df.Type=='Visual')&(df.RWD_Loc==boy_goal),
            (df.Type=='Visual')&(df.RWD_Loc==egg_goal),
            (df.Type=='Auditory')&(df.RWD_Loc==boy_goal),
            (df.Type=='Auditory')&(df.RWD_Loc==egg_goal),
            (df.Type=='Elementary')&(df.RWD_Loc==boy_goal),
            (df.Type=='Elementary')&(df.RWD_Loc==egg_goal)]
    
    cell_full_name = cell_name.strip('.csv')
    
    fr_mean = np.zeros((8,95))
    fr_sem = np.zeros((8,95))
    for i in range(6):
        fr_mean[i,:] = df[cond[i]].iloc[:,fr_id:fr_id+95].to_numpy().mean(axis=0)
        fr_sem[i,:] = stats.sem(df[cond[i]].iloc[:,fr_id:fr_id+95].to_numpy())
    
    fr_mean[6,:] = (fr_mean[0,:]+fr_mean[2,:]+fr_mean[4,:])/3    # mean firing rates for BOY
    fr_mean[7,:] = (fr_mean[1,:]+fr_mean[3,:]+fr_mean[5,:])/3    # mean firing rates for EGG
    
    fr_sem[6,:] = (fr_sem[0,:]+fr_sem[2,:]+fr_sem[4,:])/3    # SEM for BOY
    fr_sem[7,:] = (fr_sem[1,:]+fr_sem[3,:]+fr_sem[5,:])/3    # SEM for EGG
    
    if smooth:
        for i in range(8):
            fr_mean[i,:] = gaussian_filter(fr_mean[i,:],sigma=gauss_sigma)
            fr_sem[i,:] = gaussian_filter(fr_sem[i,:],sigma=gauss_sigma)
            
    y_max = np.ceil(np.max(fr_mean+fr_sem))
    y_min = np.ceil(np.abs(np.min(fr_mean-fr_sem)))*-1
    
    fig,ax = plt.subplots(2,2,figsize=(7,5))
    plt.suptitle(cell_full_name,fontsize=15);
    x = np.arange(95)*10
    
    # Mean firing rates of each object
    object_max = np.ceil(np.max(fr_mean[6:7,:]+fr_sem[6:7,:]))
    object_min = np.ceil(np.abs(np.max(fr_mean[6:7,:]-fr_sem[6:7,:])))*-1
    
    ax[0,0].plot(x,fr_mean[6,:],color='black',linewidth=linewidth)
    ax[0,0].fill_between(x,fr_mean[6,:]-fr_sem[6,:],fr_mean[6,:]+fr_sem[6,:],color='black',alpha=0.2)
    ax[0,0].plot(x,fr_mean[7,:],color='black',linewidth=linewidth,linestyle=':')
    ax[0,0].fill_between(x,fr_mean[7,:]-fr_sem[7,:],fr_mean[7,:]+fr_sem[7,:],color='black',alpha=0.2)        
    ax[0,0].scatter(object_bin*10,np.tile(object_max-0.1,(len(object_bin),1)),color='black',marker='*',s=7)
    ax[0,0].set_title('Objects',fontsize=14)    
    ax[0,0].set_yticks(np.arange(object_min,object_max+0.1,1))
    ax[0,0].set_ylim([object_min,object_max])
    ax[0,0].set_ylabel('z-scored FR',fontsize=13)
    ax[0,0].set_xlabel('Time (ms)',fontsize=13)  
    ax[0,0].set_xticks([0,400,950])
    ax[0,0].set_xlim([0,950])
    
    # Mean firing rates of all conditions

    for i in range(6):
        if i%2==0:
            ls = linestyle[0]
        else:
            ls = linestyle[1]          
        ax[1,0].plot(x,fr_mean[i,:],color=color[divmod(i,2)[0]],linewidth=linewidth,linestyle=ls)
        ax[1,0].fill_between(x,fr_mean[i,:]-fr_sem[i,:],fr_mean[i,:]+fr_sem[i,:],color=color[divmod(i,2)[0]],alpha=0.2)
    ax[1,0].scatter(object_bin*10,np.tile(y_max-0.1,(len(object_bin),1)),color='black',marker='*',s=7)
    ax[1,0].set_yticks(np.arange(y_min,y_max+0.1,1))
    ax[1,0].set_ylim([y_min,y_max])
    ax[1,0].set_ylabel('z-scored FR',fontsize=13)
    ax[1,0].set_xlabel('Time (ms)',fontsize=13)  
    ax[1,0].set_xticks([0,400,950])
    ax[1,0].set_xlim([0,950])
    
    #
    x = ['Boy-M','Boy-V','Boy-A','Egg-M','Egg-V','Egg-A']
    ax[0,1].bar(x,cond_mFR,xerr=cond_sem,yerr=cond_sem,
                color=['purple','blue','red','purple','blue','red'])
    ax[0,1].set_ylabel('mean FR',fontsize=13)
    ax[0,1].set_xticklabels(x, rotation=45, ha='right');
    ax[0,1].set_yticks(np.arange(y_min,y_max+0.1,1))
    ax[0,1].set_ylim([y_min,y_max])
    
    x = ['Boy-image','Boy-sound','Boy-int','Egg-image','Egg-sound','Egg-int']
    sig = pval<0.05
    ax[1,1].bar(x,beta_coef,
                    color=['tab:cyan','tab:pink','tab:brown','tab:cyan','tab:pink','tab:brown'])
    ax[1,1].scatter(np.where(sig)[0],np.tile(0.1,(sum(sig),1)),marker='*',color='black')
    ax[1,1].set_ylabel('beta coefficient',fontsize=13)
    ax[1,1].set_xticklabels(x, rotation=45, ha='right');
    
    plt.tight_layout()
    
    if save:
        fig_path = save_path / region / ('response' if response else '')
        os.makedirs(fig_path, exist_ok=True)

        if save_format == 'png':
            plt.savefig(fig_path / f'{cell_full_name}.png', dpi=100, facecolor='white')
        elif save_format == 'svg':
            plt.savefig(fig_path / f'{cell_full_name}.svg')
        plt.close()

In [8]:
def save_result(f):
    """
    This function saves regression results into HDF5 format.
    """
    cell_group = f.create_group(str(cell_id))
    basic_group = f.create_group(f'{cell_id}/basic')
    extended_group = f.create_group(f'{cell_id}/extended')
    basic_shuffle_group = f.create_group(f'{cell_id}/basic_shuffle')
    extended_shuffle_group = f.create_group(f'{cell_id}/extended_shuffle')
    
    basic_group.create_dataset('beta_coef',data=beta_coef)
    basic_group.create_dataset('pval',data=pval)
    basic_group.create_dataset('AIC',data=AIC)
    basic_group.create_dataset('BIC',data=BIC)

    extended_group.create_dataset('beta_coef',data=beta_coef_choice)
    extended_group.create_dataset('pval',data=pval_choice)    
    extended_group.create_dataset('AIC',data=AIC_choice)
    extended_group.create_dataset('BIC',data=BIC_choice)    
    
    basic_shuffle_group.create_dataset('beta_coef',data=beta_coef_shuffle)
    basic_shuffle_group.create_dataset('AIC',data=AIC_shuffle)
    basic_shuffle_group.create_dataset('BIC',data=BIC_shuffle)

    extended_shuffle_group.create_dataset('beta_coef',data=beta_coef_choice_shuffle)
    extended_shuffle_group.create_dataset('AIC',data=AIC_choice_shuffle)
    extended_shuffle_group.create_dataset('BIC',data=BIC_choice_shuffle)           
    
    cell_group.attrs['Rat'] = rat_id
    cell_group.attrs['Region'] = region
    cell_group.attrs['Session'] = session_id
    cell_group.attrs['Response cell'] = response

In [9]:
%%time
for cell_run,cell_name in enumerate(cell_list):
    loop_start = time.time()
    # get information about the cell
    cell_info = cell_name.split('-')
    cell_id, rat_id, session_id, region = int(cell_info[0]), cell_info[1], cell_info[2], cell_info[5]
        
    # skip non object-selective cells
    if f[str(cell_id)].attrs['Object cell'] == 0:
        continue
        
    # load cell data
    df = pd.read_csv(cell_path/cell_name)
    df.drop(df[df.Correctness==0].index,inplace=True)
    df.reset_index(inplace=True,drop=True)
    df[['Visual','Auditory']] = df[['Visual','Auditory']].fillna('no')
    
    boy_goal = df.loc[df['Visual']=='Boy','RWD_Loc'].values[0]
    boy_aud = df.loc[df['RWD_Loc']==boy_goal,'Auditory'].values[0]
    
    egg_goal = df.loc[df['Visual']=='Egg','RWD_Loc'].values[0]
    egg_aud = df.loc[df['RWD_Loc']==egg_goal,'Auditory'].values[0]  
    
    df['Boy-V'] = (df['Visual'] == 'Boy').astype(int)
    df['Boy-A'] = (df['Auditory'] == boy_aud).astype(int)
    df['Egg-V'] = (df['Visual'] == 'Egg').astype(int)
    df['Egg-A'] = (df['Auditory'] == egg_aud).astype(int)
    
    df['Boy-int'] = df['Boy-V']*df['Boy-A']
    df['Egg-int'] = df['Egg-V']*df['Egg-A']
    
    df['Choice'] = (df['RWD_Loc']==boy_goal).astype(int) 
    
    fr_id = df.columns.get_loc('Var10')  # get the index of the first firing rate column
    fr = df.iloc[:,fr_id:fr_id+95].to_numpy()    # get firing rate data into array
       
    # set independent variables for multiple linear regression
    x1 = df[['Boy-V','Boy-A','Boy-int','Egg-V','Egg-A','Egg-int']].to_numpy()
    x2 = df[['Boy-V','Boy-A','Boy-int','Egg-V','Egg-A','Egg-int','Choice']].to_numpy()
    
    # load object-selective epoch (time bins)
    object_bin = np.array(f[f'{cell_id}/object_bin'])

    # get mean firing rates of object-selective epoch in each trial
    df['Object_mFR'] = df.iloc[:,fr_id+min(object_bin):fr_id+max(object_bin)+1].mean(axis=1)
    fr_id = df.columns.get_loc('Var10')  # get the index of the first firing rate column    
    fr = df['Object_mFR'].to_numpy()

    cond_mFR = [df.groupby(['Type','RWD_Loc']).mean()['Object_mFR']['Multimodal',boy_goal],
                df.groupby(['Type','RWD_Loc']).mean()['Object_mFR']['Visual',boy_goal],
                df.groupby(['Type','RWD_Loc']).mean()['Object_mFR']['Auditory',boy_goal],
                df.groupby(['Type','RWD_Loc']).mean()['Object_mFR']['Multimodal',egg_goal],
                df.groupby(['Type','RWD_Loc']).mean()['Object_mFR']['Visual',egg_goal],
                df.groupby(['Type','RWD_Loc']).mean()['Object_mFR']['Auditory',egg_goal]]  

    cond_sem = [df.groupby(['Type','RWD_Loc']).sem()['Object_mFR']['Multimodal',boy_goal],
                df.groupby(['Type','RWD_Loc']).sem()['Object_mFR']['Visual',boy_goal],
                df.groupby(['Type','RWD_Loc']).sem()['Object_mFR']['Auditory',boy_goal],
                df.groupby(['Type','RWD_Loc']).sem()['Object_mFR']['Multimodal',egg_goal],
                df.groupby(['Type','RWD_Loc']).sem()['Object_mFR']['Visual',egg_goal],
                df.groupby(['Type','RWD_Loc']).sem()['Object_mFR']['Auditory',egg_goal]]                        
    
    # multiple linear regression
    result = MLR(fr,x1,x2,0)

    beta_coef, beta_coef_choice = result[0], result[1]
    pval, pval_choice = result[2], result[3]
    AIC, AIC_choice = result[4], result[5]
    BIC, BIC_choice = result[6], result[7]
    
    # multiple linear regression (shuffling)
    shuffle_result = Parallel(n_jobs=-1)(delayed(MLR)(fr,x1,x2,1) for i in range(1000))   
    
    beta_coef_shuffle = np.array([r[0] for r in shuffle_result])
    beta_coef_choice_shuffle = np.array([r[1] for r in shuffle_result])    
    AIC_shuffle = np.array([r[4] for r in shuffle_result])
    AIC_choice_shuffle = np.array([r[5] for r in shuffle_result])      
    BIC_shuffle = np.array([r[6] for r in shuffle_result])
    BIC_choice_shuffle = np.array([r[7] for r in shuffle_result])

    # define response-selective neurons by model improvement after adding response term
    model_improv = AIC-AIC_choice
    if model_improv > np.percentile(AIC_shuffle-AIC_choice_shuffle,99):
        response = 1
    else:
        response = 0    
        
    plot_SDF_beta(df,2,1,1,'png')
    
    # save results into HDF5 format
    save_result(s)
            
    loop_end = time.time()
    loop_time = divmod(loop_end-loop_start,60)
    print(cell_name.strip('.csv'), f'////// {cell_run+1}/{len(cell_list)} completed  //////  {int(loop_time[0])} min {loop_time[1]:.2f} sec')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33588\3917152087.py:80: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[0,1].set_xticklabels(x, rotation=45, ha='right');
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33588\3917152087.py:88: MatplotlibDeprecationWarning: Support for passing numbers through unit converters is deprecated since 3.5 and support will be removed two minor releases later; use Axis.convert_units instead.
  ax[1,1].scatter(np.where(sig)[0],np.tile(0.1,(sum(sig),1)),marker='*',color='black')
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33588\3917152087.py:90: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[1,1].set_xticklabels(x, rotation=45, ha='right');


0004-600-1-1-Crossmodal-TeV-deep-(-7.32 mm)-TT4.2 ////// 2/878 completed  //////  0 min 6.44 sec


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33588\3917152087.py:80: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[0,1].set_xticklabels(x, rotation=45, ha='right');
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33588\3917152087.py:90: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[1,1].set_xticklabels(x, rotation=45, ha='right');


0006-600-1-1-Crossmodal-PER-superficial-(-7.2 mm)-TT5.1 ////// 4/878 completed  //////  0 min 1.17 sec


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33588\3917152087.py:80: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[0,1].set_xticklabels(x, rotation=45, ha='right');
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33588\3917152087.py:88: MatplotlibDeprecationWarning: Support for passing numbers through unit converters is deprecated since 3.5 and support will be removed two minor releases later; use Axis.convert_units instead.
  ax[1,1].scatter(np.where(sig)[0],np.tile(0.1,(sum(sig),1)),marker='*',color='black')
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33588\3917152087.py:90: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[1,1].set_xticklabels(x, rotation=45, ha='right');


...

2406-699-6-6-Crossmodal-Visual-deep-(-6.12 mm)-TT21.2 ////// 876/878 completed  //////  0 min 1.10 sec


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33588\3917152087.py:80: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[0,1].set_xticklabels(x, rotation=45, ha='right');
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33588\3917152087.py:90: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[1,1].set_xticklabels(x, rotation=45, ha='right');


2407-699-6-6-Crossmodal-Visual-deep-(-6.12 mm)-TT21.3 ////// 877/878 completed  //////  0 min 1.30 sec


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33588\3917152087.py:80: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[0,1].set_xticklabels(x, rotation=45, ha='right');
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33588\3917152087.py:90: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[1,1].set_xticklabels(x, rotation=45, ha='right');


2409-699-6-6-Crossmodal-Visual-superficial-(-6.36 mm)-TT23.1 ////// 878/878 completed  //////  0 min 1.07 sec
CPU times: total: 6min 53s
Wall time: 10min


In [10]:
f.close()
s.close()
print('END')

END
